In [15]:
import numpy as np
import json
import re

In [16]:
# 读取 .npy 文件，允许加载包含 Python 对象的数组
data_out = np.load("result.npy", allow_pickle=True)
data_out[39]

{'text_generation_text': ["<|User|>:Here is a question about formal logic, the correct answer is one of the options A/B/C/D. Please select the correct option and answer the question with 'The right option is'.\nQuestion: Select the best translation into predicate logic. Leo is taller than Cathy. (c: Cathy; l: Leo; Txy: x is taller than y) \nA.Tlc\nB.Tcl\nC.lTc\nD.cTl<eoh>\n<|Bot|>:Let's think step by step,'The right option is A.' diamonds.'.'"]}

In [17]:
out_list = []
for index, item in enumerate(data_out):
    # try:
    # 尝试在数据中匹配正则表达式
    match_out = re.search(r"The right option is ([A-D])", item.get('text_generation_text', [''])[0])
    if match_out:
        # if index not in [2852, 2882]:
        out_list.append(match_out.group(1))
        # else:
            # print(f"{index} is skipped ------------------------------------->> !")
        # print(f"Extracted option: {match_out.group(1)} at index {index}")
        # out_list.append(match_out.group(1))
    else:
        # 匹配失败，输出索引号
        print(f"No match found at index {index}: {item.get('text_generation_text', [''])[0]}")
        out_list.append('C')
        print("write : _________________A______________________")
#     except Exception as e:
#         # 捕获其他异常并输出索引号和错误
#         print(f"Error at index {index}: {item}. Error: {str(e)}")
# out_list

In [18]:
# 读取JSON文件
with open('/mnt/finetune/mmlu/mmlu_alpaca_format2000.json', 'r', encoding='utf-8') as f:
    data_check = json.load(f)

In [19]:
check_list = []
for index, item in enumerate(data_check):
    try:
        # 进行正则匹配
        match_check = re.search(r"The right option is ([A-D]).", item["output"])
        if match_check:
            check_list.append(match_check.group(1))
        else:
            # 匹配失败时，输出索引号和对应内容
            print(f"No match found at index {index}: {item['output']}")
    except Exception as e:
        # 捕获并显示异常以及索引号
        print(f"Error at index {index}: {item}. Error: {str(e)}")


In [20]:
print(len(out_list))
print(len(check_list))

2000
2000


In [21]:
# 使用列表推导式来比较两个列表的元素
result = [option1 == option2 for option1, option2 in zip(out_list,check_list)]
def calculate_accuracy(results):
    correct_count = sum(results)
    total_count = len(results)
    accuracy = correct_count / total_count if total_count > 0 else 0
    return accuracy
# 计算准确率
accuracy = calculate_accuracy(result)
print(f"Accuracy: {accuracy * 100:.2f}%")  # 输出准确率

Accuracy: 99.30%


In [22]:
# 使用列表推导式获取值为 False 的索引
false_indices = [index for index, value in enumerate(result) if not value]

print(false_indices)
print(len(false_indices))


[98, 227, 425, 480, 563, 652, 685, 1044, 1049, 1767, 1798, 1894, 1896, 1955]
14


In [23]:
for item in false_indices:
    print('index : ',item)
    print(data_out[item])
    print(data_check[item])
    

index :  98
{'text_generation_text': ["<|User|>:Here is a question about global facts, the correct answer is one of the options A/B/C/D. Please select the correct option and answer the question with 'The right option is'.\nQuestion: Controlling for inflation and PPP-adjustment, about how much did GDP per capita increase from 1950 to 2016 in South Korea? \nA.by 4 fold\nB.by 8 fold\nC.by 16 fold\nD.by 32 fold<eoh>\n<|Bot|>:Let's think step by step,'The right option is C.' individuals. diamonds...."]}
{'instruction': "Here is a question about global facts, the correct answer is one of the options A/B/C/D. Please select the correct option and answer the question with 'The right option is'.", 'input': 'Question: Controlling for inflation and PPP-adjustment, about how much did GDP per capita increase from 1950 to 2016 in South Korea? \nA.by 4 fold\nB.by 8 fold\nC.by 16 fold\nD.by 32 fold', 'output': 'The right option is D.'}
index :  227
{'text_generation_text': ["<|User|>:Here is a question

In [24]:
def extract_random_entries(input_file, output_file, false_indices):
    try:
        # 从输入 JSON 文件中读取内容
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        extracted_data = [] 
        # 使用 false_indices 从 JSON 数据中获取对应的内容
        for index in false_indices:
            extracted_data.append(data[index])
            
        # 将抽取的数据写入输出 JSON 文件
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(extracted_data, f, ensure_ascii=False, indent=4)

        print(f"成功抽取 {len(extracted_data)} 条数据并保存到 {output_file}")
    except Exception as e:
        print(f"发生错误: {e}")

In [25]:
# extract_random_entries('/mnt/finetune/mmlu/mmlu_alpaca_format.json','/mnt/finetune/mmlu/false_result.json',false_indices)

In [26]:
extract_random_entries('/mnt/finetune/mmlu/mmlu_alpaca_format.json','/mnt/finetune/mmlu/false_deal_math_cot_result.json',false_indices)

成功抽取 14 条数据并保存到 /mnt/finetune/mmlu/false_deal_math_cot_result.json


In [27]:
def classify_entries_by_instruction(input_file):
    try:
        # 读取输入 JSON 文件
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 正则表达式匹配 "Here is a question about ..." 并提取后续内容进行分类
        pattern = re.compile(r'Here is a question about (.*?),')
        classifications = {}

        for entry in data:
            match = pattern.search(entry['instruction'])
            if match:
                category = match.group(1).strip()
                if category not in classifications:
                    classifications[category] = []
                classifications[category].append(entry)

        # 打印分类结果，使用固定宽度对齐
        max_category_length = max(len(category) for category in classifications.keys())
        for category, entries in classifications.items():
            print(f"Category: {category.ljust(max_category_length)} | Number of Entries: {len(entries)}")

    except Exception as e:
        print(f"发生错误: {e}")

In [28]:
classify_entries_by_instruction('/mnt/finetune/mmlu/false_deal_math_cot_result.json')

Category: college medical statistics | Number of Entries: 1
Category: jurisprudence              | Number of Entries: 1
Category: agronomy                   | Number of Entries: 2
Category: anatomy                    | Number of Entries: 1
Category: ancient chinese            | Number of Entries: 2
Category: astronomy                  | Number of Entries: 2
Category: chinese food culture       | Number of Entries: 1
Category: chinese foreign policy     | Number of Entries: 1
Category: chinese history            | Number of Entries: 3
